This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [ ]:
from pydrake.all import ModelVisualizer, StartMeshcat, PackageMap, Simulator

from manipulation import running_as_notebook, ConfigureParser, FindResource
from manipulation.station import load_scenario, MakeHardwareStation

import logging
from copy import copy
from enum import Enum

import numpy as np
from pydrake.all import (
    AbstractValue,
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    InputPortIndex,
    LeafSystem,
    MeshcatVisualizer,
    Parser,
    PiecewisePolynomial,
    PiecewisePose,
    PointCloud,
    PortSwitch,
    Solve,
    RandomGenerator,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
)

from manipulation import ConfigureParser, FindResource, running_as_notebook
from manipulation.clutter import GenerateAntipodalGraspCandidate
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.pick import (
    MakeGripperCommandTrajectory,
    MakeGripperFrames,
    MakeGripperPoseTrajectory,
)
from manipulation.scenarios import AddIiwaDifferentialIK, ycb
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    add_directives,
    load_scenario,
)


class NoDiffIKWarnings(logging.Filter):
    def filter(self, record):
        return not record.getMessage().startswith("Differential IK")


logging.getLogger("drake").addFilter(NoDiffIKWarnings())


import mpld3
import numpy as np
import pydot
from IPython.display import SVG, display
from matplotlib import pyplot as plt
from pydrake.all import DiagramBuilder, StartMeshcat

from manipulation import FindResource, running_as_notebook
from manipulation.scenarios import AddMultibodyTriad
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    load_scenario,
)


if running_as_notebook:
    mpld3.enable_notebook()

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://443541ce-e170-4459-8340-438ed0837988.deepnoteproject.com/7001/


In [ ]:
def AddSpotRemote(parser):
    parser.package_map().AddRemote(
        package_name="spot_description",
        params=PackageMap.RemoteParams(
            urls=[
                f"https://github.com/bdaiinstitute/spot_ros2/archive/d429947a1df842ec38f8c6099dde9501945090d6.tar.gz"
            ],
            sha256=(
                "e4dd471be4e7e822a12afcfd6a94ce7ecbb39e2d4ea406779a96e146a607bf53"
            ),
            strip_prefix="spot_ros2-d429947a1df842ec38f8c6099dde9501945090d6/spot_description/",
        ),
    )

# Simplified Spot model for mobile manipulation

First we'll use the ModelVisualizer to inspect the model.

In [ ]:
visualizer = ModelVisualizer(meshcat=meshcat)
ConfigureParser(visualizer.parser())
AddSpotRemote(visualizer.parser())
visualizer.AddModels(
    url="package://manipulation/spot/spot_with_arm_and_floating_base_actuators.urdf"
)
visualizer.Run(loop_once=not running_as_notebook)
meshcat.DeleteAddedControls()

INFO:drake:PackageMap: Downloading https://github.com/bdaiinstitute/spot_ros2/archive/d429947a1df842ec38f8c6099dde9501945090d6.tar.gz
INFO:drake:Click 'Stop Running' or press Esc to quit
==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



KernelInterrupted: Execution interrupted by the Jupyter kernel.

Now we can use HardwareStation to create a basic simulation.

In [ ]:
from pydrake.all import RgbdSensor, RandomGenerator
from manipulation.scenarios import AddRgbdSensors

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
scenario = load_scenario(
    filename=FindResource(
        "models/spot/spot_with_arm_and_floating_base_actuators.scenario.yaml"
    )
)
station = MakeHardwareStation(
    scenario, meshcat, parser_preload_callback=AddSpotRemote
)
AddRgbdSensors(builder, plant, scene_graph)
plant = station.GetSubsystemByName("plant")
simulator = Simulator(station)
context = simulator.get_mutable_context()
SVG(pydot.graph_from_dot_data(station.GetGraphvizString())[0].create_svg())
#x0 = [0 for _ in range(20)]
#station.GetInputPort("spot.desired_state").FixValue(context, x0)
#x0 = station.GetOutputPort("spot.state_estimated").Eval(context)
#color_image = station.GetOutputPort("hand.rgb_image").Eval(context)
#index = 1
#plt.figure(figsize=(6, 12))
#plt.subplot(6, 2, index)
#plt.imshow(color_image.data)
#index += 1
#plt.title("Color image")
#plt.show()

# #print(x0)
# x0[2]+=5.0
# #x0[7]+=0.1

#example: ball is at [1,0.5,0.5]
x0=[ 3. ,  0. ,  0. ,  0.,  0.,  0.,  0.,   0. ,  0. ,  0.  , 0. ,  0.  , 0. ,  0. , 0. ,  0. ,  0. ,  0. ,  0. ,  0. ]
station.GetInputPort("spot.desired_state").FixValue(context, x0)
# simulator.AdvanceTo(1.0);
#REFERENCE: https://github.com/RussTedrake/manipulation/blob/5296103e8f671444ab390b692ed1f6de8fdf420f/pose/camera_sim.ipynb#L64


In [ ]:
x0=[ 3. ,  0. ,  0. ,  0.,  0.,  0.,  0.,   0. ,  0. ,  0.  , 0. ,  0.  , 0. ,  0. , 0. ,  0. ,  0. ,  0. ,  0. ,  0. ]
print(len(x0))

20


In [ ]:
print(station.GetSystems())
for i in range(station.num_output_ports()):
   print(station.get_output_port(i).get_name())
station.GetOutputPort("hand.rgb_image").get_data_type()

[<pydrake.multibody.plant.MultibodyPlant object at 0x7f5ad097c670>, <pydrake.geometry.SceneGraph object at 0x7f5ad1593bf0>, <pydrake.systems.controllers.InverseDynamicsController object at 0x7f5abab1a430>, <pydrake.systems.sensors.RgbdSensor object at 0x7f5abaaf3f30>, <pydrake.systems.sensors.RgbdSensor object at 0x7f5abaaf0df0>, <pydrake.systems.sensors.RgbdSensor object at 0x7f5abab2e330>, <pydrake.systems.sensors.RgbdSensor object at 0x7f5abaca4f30>, <pydrake.systems.sensors.RgbdSensor object at 0x7f5abab2c230>, <pydrake.systems.sensors.RgbdSensor object at 0x7f5abab2c3b0>, <pydrake.systems.primitives.SharedPointerSystem object at 0x7f5abaac85b0>, <pydrake.geometry.DrakeVisualizer object at 0x7f5abad90df0>, <pydrake.geometry.DrakeVisualizer object at 0x7f5abad92ef0>, <pydrake.multibody.plant.ContactResultsToLcmSystem object at 0x7f5ad15adaf0>, <pydrake.systems.lcm.LcmPublisherSystem object at 0x7f5ad15acf30>, <pydrake.systems.framework.System object at 0x7f5ad15ad270>, <pydrake.geom

<PortDataType.kAbstractValued: 1>

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
def move_forward(time):
    # Assuming `station` is the hardware station
    context = simulator.get_mutable_context()

    # Get the estimated state of the Spot robot
    x0 = station.GetOutputPort("spot.state_estimated").Eval(context)

    # Modify the desired state to command forward movement
    desired_state = x0
    desired_state[7]+= 0.1  # Adjust the linear velocity as needed in the x-direction
    desired_state[0] += 1.0  # Optionally, adjust the x-position to move forward

    # Set the modified desired state
    station.GetInputPort("spot.desired_state").FixValue(context, desired_state)

    x_before_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State Before Advance:", x_before_advance)

    # Advance the simulation to see the effect
    simulator.AdvanceTo(time)  # Adjust the simulation time as needed

    x_after_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State After Advance:", x_after_advance)

def move_backward(time):
    # Assuming `station` is the hardware station
    context = simulator.get_mutable_context()

    # Get the estimated state of the Spot robot
    x0 = station.GetOutputPort("spot.state_estimated").Eval(context)

    # Modify the desired state to command forward movement
    desired_state = x0
    desired_state[7] -= 0.1  # Adjust the linear velocity as needed in the x-direction
    desired_state[0] -= 1.0  # Optionally, adjust the x-position to move forward

    # Set the modified desired state
    station.GetInputPort("spot.desired_state").FixValue(context, desired_state)

    x_before_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State Before Advance:", x_before_advance)

    # Advance the simulation to see the effect
    simulator.AdvanceTo(time)  # Adjust the simulation time as needed

    x_after_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State After Advance:", x_after_advance)

def move_left(time):
    # Assuming `station` is the hardware station
    context = simulator.get_mutable_context()

    # Get the estimated state of the Spot robot
    x0 = station.GetOutputPort("spot.state_estimated").Eval(context)

    # Modify the desired state to command forward movement
    desired_state = x0.copy()
    desired_state[8] = 0.1  # Adjust the linear velocity as needed in the x-direction
    desired_state[1] += 1.0  # Optionally, adjust the x-position to move forward

    # Set the modified desired state
    station.GetInputPort("spot.desired_state").FixValue(context, desired_state)

    x_before_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State Before Advance:", x_before_advance)

    # Advance the simulation to see the effect
    simulator.AdvanceTo(time)  # Adjust the simulation time as needed

    x_after_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State After Advance:", x_after_advance)

def move_right(time):
    # Assuming `station` is the hardware station
    context = simulator.get_mutable_context()

    # Get the estimated state of the Spot robot
    x0 = station.GetOutputPort("spot.state_estimated").Eval(context)

    # Modify the desired state to command forward movement
    desired_state = x0
    desired_state[8] -= 0.1  # Adjust the linear velocity as needed in the x-direction
    desired_state[1] -= 1.0  # Optionally, adjust the x-position to move forward

    # Set the modified desired state
    station.GetInputPort("spot.desired_state").FixValue(context, desired_state)

    x_before_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State Before Advance:", x_before_advance)

    # Advance the simulation to see the effect
    simulator.AdvanceTo(time)  # Adjust the simulation time as needed

    x_after_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State After Advance:", x_after_advance)

def rotate_base(time):
    # Assuming `station` is the hardware station
    context = simulator.get_mutable_context()

    # Get the estimated state of the Spot robot
    x0 = station.GetOutputPort("spot.state_estimated").Eval(context)

    # Modify the desired state to command forward movement
    desired_state = x0
    desired_state[10] += 0.1  # Adjust the linear velocity as needed in the x-direction
    desired_state[3] += 1.0  # Optionally, adjust the x-position to move forward

    # Set the modified desired state
    station.GetInputPort("spot.desired_state").FixValue(context, desired_state)

    x_before_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State Before Advance:", x_before_advance)

    # Advance the simulation to see the effect
    simulator.AdvanceTo(time)  # Adjust the simulation time as needed

    x_after_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State After Advance:", x_after_advance)

def extend_arm(time):
    # Assuming `station` is the hardware station
    context = simulator.get_mutable_context()

    # Get the estimated state of the Spot robot
    x0 = station.GetOutputPort("spot.state_estimated").Eval(context)

    # Modify the desired state to command forward movement
    desired_state = x0
    desired_state[9] += 0.1  # Adjust the linear velocity as needed in the x-direction
    desired_state[2] += 1.0  # Optionally, adjust the x-position to move forward

    # Set the modified desired state
    station.GetInputPort("spot.desired_state").FixValue(context, desired_state)

    x_before_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State Before Advance:", x_before_advance)

    # Advance the simulation to see the effect
    simulator.AdvanceTo(time)  # Adjust the simulation time as needed

    x_after_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State After Advance:", x_after_advance)



def arm_basejoint(time):
    # Assuming `station` is the hardware station
    context = simulator.get_mutable_context()

    # Get the estimated state of the Spot robot
    x0 = station.GetOutputPort("spot.state_estimated").Eval(context)

    # Modify the desired state to command forward movement
    desired_state = x0
    desired_state[10] += 0.1  # Adjust the linear velocity as needed in the x-direction
    desired_state[4] += 1.0  # Optionally, adjust the x-position to move forward

    # Set the modified desired state
    station.GetInputPort("spot.desired_state").FixValue(context, desired_state)

    x_before_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State Before Advance:", x_before_advance)

    # Advance the simulation to see the effect
    simulator.AdvanceTo(time)  # Adjust the simulation time as needed

    x_after_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State After Advance:", x_after_advance)


def arm_wristjoint(time):
    # Assuming `station` is the hardware station
    context = simulator.get_mutable_context()

    # Get the estimated state of the Spot robot
    x0 = station.GetOutputPort("spot.state_estimated").Eval(context)

    # Modify the desired state to command forward movement
    desired_state = x0
    desired_state[11] += 0.1  # Adjust the linear velocity as needed in the x-direction
    desired_state[5] += 1.0  # Optionally, adjust the x-position to move forward

    # Set the modified desired state
    station.GetInputPort("spot.desired_state").FixValue(context, desired_state)

    x_before_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State Before Advance:", x_before_advance)

    # Advance the simulation to see the effect
    simulator.AdvanceTo(time)  # Adjust the simulation time as needed

    x_after_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State After Advance:", x_after_advance)


def arm_rotation(time):
    # Assuming `station` is the hardware station
    context = simulator.get_mutable_context()

    # Get the estimated state of the Spot robot
    x0 = station.GetOutputPort("spot.state_estimated").Eval(context)

    # Modify the desired state to command forward movement
    desired_state = x0
    desired_state[13] += 0.1  # Adjust the linear velocity as needed in the x-direction
    desired_state[6] += 1.0  # Optionally, adjust the x-position to move forward

    # Set the modified desired state
    station.GetInputPort("spot.desired_state").FixValue(context, desired_state)

    x_before_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State Before Advance:", x_before_advance)

    # Advance the simulation to see the effect
    simulator.AdvanceTo(time)  # Adjust the simulation time as needed

    x_after_advance = station.GetOutputPort("spot.state_estimated").Eval(context)
    print("State After Advance:", x_after_advance)



In [ ]:
from pydrake.all import InverseKinematics

# Create the InverseKinematics object
plant_context = plant.GetMyMutableContextFromRoot(context)
ik = InverseKinematics(plant, plant_context)
 
# Set the target frame for the end effector
end_effector_frame = plant.GetFrameByName("arm_link_fngr")
ik.AddPositionConstraint(
    frameB=end_effector_frame,
    p_BQ=np.zeros(3),  # Desired position in the end effector frame
    frameA=plant.world_frame(),
    p_AQ_upper = [1,1,1], 
    p_AQ_lower = [-1,-1,0])

# Set the initial guess for joint positions
q_init_guess = np.zeros(plant.num_positions())

# Solve the IK problem
prog = ik.get_mutable_prog()
q = ik.q()
prog.SetInitialGuess(q, q_init_guess)
result = Solve(ik.prog())

if result.is_success():
    # Get the solution
    q_solution = result.GetSolution()
    print("IK solution:", q_solution)
else:
    print("IK failed to converge.")



#builder = DiagramBuilder()
#plant_system = builder.AddSystem(plant)
#ConnectMeshcatVisualizer(builder, plant.get_scene_graph(), output_port=plant.get_visualization_output_port())
#diagram = builder.Build()

#simulator = Simulator(diagram)
#context = simulator.get_mutable_context()
#plant_context = plant.GetMyMutableContextFromRoot(context)
#plant.SetPositions(plant_context, q_solution)

#simulator.AdvanceTo(1.0)

IK solution: [-0.14992595  0.          0.          0.         -0.01346539  0.
  0.         -0.00222175  0.          0.        ]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=443541ce-e170-4459-8340-438ed0837988' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>